## Wiener Diarum Sterbelisten
-----------------------------------------------------------------------------------------------

### Data

The digitalised Wiener Zeitung is partially available as TEI XML as well as an extracted HTML on Github. Starting from 1703, there is representative data.This data set contains only a few newspapers per year starting from 1706.

### Task

Filter out Toponyms from the Sterbelisten. For this sequence labeling task we need to first tokenize the words and use the rules (patterns and keywords) made by the team (Nina) to fit to our classes (Toponym or not).


##### What we have 
There is a list with sucessfully extracted toponyms, which are already normalised orthography.
There is a list with rules which are combines Regex and other expressions. 
There is an html file that has \<mark> on most toponyms but sometimes on persons and other names too.
There are defined classes (Toponym/no Toponym).
There are .csv files with Toponyms extracted from Wien Geschichte Wiki.

##### What we want
A trainig and test set containing three matrixes:
T Matrix containing the classes and the rules.(np.matrix, sparse)
Z Matrix containing the samples and the rules.(np.matrix, sparse)
X Matrix containing the samples (data frame)


##### What to do

**1. Read in the html file**
Strip of tags and clean the text with re
create a list with each sentence as an item
create a list with each word as an item
create a data frame
colums= words /lb
rows = sentenses

**2. Read in the patterns**
clean the patterns 
create a set from patterns
DONE

**3. Read in the file with already extracted placenames**
create a set from exsiting placenames
Extract all \<mark> words from HTML**
Using their code
DONE

**4. Building training data**
As there is no clean manual training data, it has to be build:


    create a counter = 0 ## just as controll

    for each row in the df 
        if word in row matches patterns:
            if it contains \<mark>:
                keep \<mark>
                remove all other marks from data frame (like on person or Names)
                add 1 to colum "Toponym"
        elif:
            for each word in row:
                compare to list with placenames
                if it is in list:
                    if Toponym not 1:
                    add 1 to colum "Toponym"
        counter+=1

    splitting into train & dev data


**5. Building test data** 

manually checked! 
There is the 'timemachine_evaluatuin_v1_edited_corrected.jsonl which should contain manually corrected place names but I don't understand where the actual word is tagged or how to filter out that information.

### 1. Read in HTML

In [2]:
'''Import libraries and load html file'''

import os
import re
import codecs

file = codecs.open('annotations_3-21_v4.html', "r", "utf-8")
sterbelisten_html = file.read()


'''Cleaning the text first from both html taggs and xml tags and than in a second step clean the intro and replace with /n'''

sterbelisten_strip_html_1 = re.sub(r"<h2>.*</h2><h3>.*xml \|.*\d+</h3><br/>","",sterbelisten_html)

sterbelisten_strip_html1 = re.sub(r"<hr/>|<p>|</p>|#+|=| =|<html>|</html>|\r|/|\b \.|\b# \.","",sterbelisten_strip_html_1) #### except for <mark>|</mark>

sterbelisten_strip_html2 = re.sub(r"    \b","",sterbelisten_strip_html1)

In [3]:
'''create an empty list and split the text at line break, creating von element for each sentense'''

sterbeliste = []
sterbeliste = sterbelisten_strip_html2.split("\n\n\n")
#print(sterbeliste)

'''create a list with words for each sentence, but shoud leave out <mark>'''
sterbeliste_words= [i.split(" ") for i in sterbeliste if i!= "<mark>"]## still splits my words with mark

'''removing empty objects in sterbeliste_words'''

#sterbeliste_words = [word for sentense in sterbeliste_words list(filter(None, i))]
#sterbeliste_words= [i for i in sterbeliste_words if i!=' ']
#sterbeliste_words= [word for word in sterbeliste_words if not "re.search(\s)"]
'''Problems'''
### find a way to filter out the rest of the "" elements in the lists. 
### find a way to have the <mark> <mark> words together

'Problems'

## 2. Read in Patterns

In [5]:
'''rules by Regex/Patterns'''
'''Importing the patterns from sterbelisten GitHub'''

import pandas as pd
import jsonlines as jsl

with jsl.open('patterns.jsonl','r') as reader: #read in jsonl with jsonlines
    df = pd.json_normalize(reader, record_path =['pattern']) 
    # create data frame only using REGEX from the patterns

patterns_list = df["TEXT.REGEX"].tolist() # making a list of all REGEX patterns
patterns = set(patterns_list) # making a set, so that every REGEX expression is unique
patterns = {x for x in patterns if pd.notna(x)} # make sure no NaN are in the set
#print(patterns)
print(len(patterns))

10


## 3. Create list with keywords from HTML

In [7]:
'''rules by using Keywords'''

''' Build abbreviations list, with compiled regexes -> from sterbelisten GITHUB'''

import csv

def build_abbr_replacement_regex(abbr):
    return re.compile(r'\b' + re.escape(abbr) + r'\b')

with open(os.path.join(".", "Abk_Liste.csv"),encoding='utf-8') as f:
    abbreviations_file = csv.DictReader(f)

    # Build a dict of abbreviations, values and compiled regexes (which we will need to do replacements)
    abbreviations = [
        {"abbreviation": row["Abkürzung"], "expansion": row["Bedeutung"], "regex": build_abbr_replacement_regex(row["Abkürzung"]) } 
        for row in abbreviations_file if row["Bedeutung"]
    ]


'''Work on how to actually replace words with the expansion'''
#for i in abbreviations:
#    print(i)
    #new_sterbeliste= i["regex"].sub(i["expansion"], sterbeliste_words)
    
    
#for i in sterbeliste: 
#    for word in abbreviations:
#        if word['abbreviation'] in i:
#            build_abbr_replacement_regex(i)            
            

'Work on how to actually replace words with the expansion'

In [9]:
from io import StringIO
import os
from lxml import etree

""" Get all the possible toponyms from annotations*.html file """

file_path = os.path.join(".", "annotations_3-21_v4.html")

# The annotations document is not well-formed as XML, so we need to use the HTMLParser
parser = etree.HTMLParser()

with open(file_path, encoding='utf-8') as f:
    raw_html = f.read()

# Parse the raw_html
tree = etree.parse(StringIO(raw_html), parser)

root = tree.getroot()
marks = root.xpath("//mark")

length = len(marks)

# Construct a set of toponyms (thus removing duplicates)
toponym_strings = {mark.text for mark in marks}



In [11]:
'''Create a list with all words that were marked with mark'''

toponymstring=[]

# clean the strings-> does no work yet
for mark in marks:
        toponym= re.sub(r"#+|=| =|-|/|\.|\d+\s+J","",mark.text) ### why does this not filter out f.ex. "Laimgr   50  J"  
        toponymstring.append(toponym)
print(len(toponymstring))

# create a set so that the iteration later is easier
toponym_set= set(toponymstring)
len(toponym_set)

14870


7226

## 4.Building training data

In [37]:
### create a tokenized sentenses with spacy
import spacy 
from spacy.lang.de.examples import sentences

nlp = spacy.load("de_core_news_sm")

for i in sterbeliste:
    doc = nlp(i)
    print(doc.text)
    for token in doc:
        print(token.text, token.pos_, token.dep_)


Dem Peter Frost  einem Cammer im <mark>Greiseckeris  Hauß<mark> im <mark>Diener  Gäßl<mark>  sein Kind Frantz  alt 6 viertl Jahr 

 SPACE dep
Dem DET nk
Peter PROPN pnc
Frost PROPN ROOT
  SPACE dep
einem DET nk
Cammer NOUN da
im ADP mnr
< ADJ nk
mark NOUN nk
> X ROOT
Greiseckeris PROPN pnc
  SPACE dep
Hauß PROPN pnc
< ADJ nk
mark NOUN ROOT
> X nk
im ADP ROOT
< ADJ nk
mark NOUN nk
> X ROOT
Diener ADJ nk
  SPACE dep
Gäßl PROPN pnc
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
sein DET nk
Kind NOUN ROOT
Frantz PROPN pnc
  SPACE dep
alt ADV mo
6 NUM nk
viertl ADJ nk
Jahr NOUN pd
Der Maria Nauitschanin  einer Burgerl Wittib im <mark>Primis  Hauß<mark> auf der <mark>Wendelstadt<mark>  ihr Kind Carl  alt 5 Jahr 
Der DET nk
Maria PROPN pnc
Nauitschanin PROPN da
  SPACE dep
einer DET nk
Burgerl NOUN ROOT
Wittib NOUN ROOT
im ADP mnr
< ADJ nk
mark NOUN nk
> X punct
Primis PROPN ROOT
  SPACE dep
Hauß PROPN ROOT
< ADJ nk
mark NOUN mnr
> X nk
auf ADP mnr
der DET nk
< ADJ nk
mark NOUN nk
> X punct
W

Dem Hannß Binder  Burgerlichen Landkutschern  beym <mark>guldenen Apffel<mark> auff der <mark>Widen<mark>  sein Kind Michael  alt 6 Viertl Jahr 
Dem DET nk
Hannß ADJ nk
Binder NOUN ROOT
  SPACE dep
Burgerlichen ADJ nk
Landkutschern NOUN oa
  SPACE dep
beym VERB mo
< ADJ nk
mark NOUN sb
> X nk
guldenen ADJ ROOT
Apffel NOUN ROOT
< ADJ nk
mark NOUN nk
> X punct
auff PROPN ROOT
der DET nk
< ADJ nk
mark NOUN punct
> X ROOT
Widen NOUN nk
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
sein DET nk
Kind NOUN ROOT
Michael PROPN pnc
  SPACE dep
alt ADV mo
6 NUM nk
Viertl NOUN pnc
Jahr NOUN ROOT
Joseph Schmidt  ein Laquey beym <mark>guldenen Väßl<mark> bey <mark>St  Ulrich<mark>  alt 30 Jahr 
Joseph PROPN pnc
Schmidt PROPN ROOT
  SPACE dep
ein DET nk
Laquey NOUN ROOT
beym VERB mnr
< PROPN nk
mark NOUN oa
> X nk
guldenen ADJ ROOT
Väßl PROPN pnc
< ADJ nk
mark NOUN ROOT
> X nk
bey X ROOT
< ADJ nk
mark NOUN nk
> X ROOT
St X pnc
  SPACE dep
Ulrich PROPN pnc
< PROPN nk
mark NOUN ROOT
> X punct
  SPACE de

Dem Georg Zis  Laggey  im <mark>Brunmeisterischen Hauß<mark> auff der <mark>Laimbgruben<mark>  sein Weib Catharina  alt 70 Jahr 
Dem DET nk
Georg PROPN pnc
Zis PROPN pnc
  SPACE dep
Laggey PROPN nk
  SPACE dep
im ADP mnr
< ADJ nk
mark NOUN nk
> X ROOT
Brunmeisterischen PROPN nk
Hauß PROPN pnc
< ADJ nk
mark NOUN ROOT
> X nk
auff PROPN ROOT
der DET nk
< ADJ nk
mark NOUN punct
> X punct
Laimbgruben NOUN ROOT
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
sein DET nk
Weib NOUN ROOT
Catharina PROPN pnc
  SPACE dep
alt ADV mo
70 NUM nk
Jahr NOUN ROOT
Dem Mathias Durner  Leinweber  beym <mark>gueten Hürten<mark> bey <mark>Maria  Hülff<mark>  sein Kind Susanna  alt 6 Jahr 
Dem DET nk
Mathias PROPN pnc
Durner PROPN ROOT
  SPACE dep
Leinweber PROPN oa
  SPACE dep
beym VERB ROOT
< ADJ nk
mark NOUN sb
> X nk
gueten ADJ nk
Hürten NOUN da
< ADJ nk
mark NOUN nk
> X punct
bey X ROOT
< ADJ nk
mark NOUN nk
> X ROOT
Maria PROPN pnc
  SPACE dep
Hülff PROPN ROOT
< ADJ nk
mark NOUN nk
> X punct
  SPACE dep
s

Max Christel  Herrn Taffeldecker  beym <mark>rothen Gattern<mark> unterm <mark>Land  Hauß<mark>  alt 36 Jahr 
Max PROPN pnc
Christel PROPN pnc
  SPACE dep
Herrn NOUN sb
Taffeldecker PROPN nk
  SPACE dep
beym VERB ROOT
< PROPN nk
mark NOUN oa
> X nk
rothen ADJ nk
Gattern NOUN ROOT
< ADJ nk
mark NOUN nk
> X ROOT
unterm ADP ROOT
< ADJ nk
mark NOUN nk
> X ROOT
Land NOUN ROOT
  SPACE dep
Hauß PROPN pnc
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
alt ADV mo
36 NUM nk
Jahr NOUN ROOT
Dem Johann Osterman  Kayserl Burgg Nachtwachter  im <mark>Reschgallischen Hauß<mark> im <mark>Ballgäßl<mark>  sein Kind Antoni  alt 3 viertl Jahr 
Dem DET nk
Johann PROPN pnc
Osterman PROPN pnc
  SPACE dep
Kayserl NOUN ROOT
Burgg PROPN nk
Nachtwachter NOUN ROOT
  SPACE dep
im ADP mnr
< ADJ nk
mark NOUN nk
> X punct
Reschgallischen NOUN pnc
Hauß PROPN ROOT
< ADJ nk
mark NOUN pnc
> X nk
im ADP ROOT
< ADJ nk
mark NOUN nk
> X ROOT
Ballgäßl PROPN pnc
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
sein DET nk
Kind NOUN

Laimbl NOUN ROOT
< ADJ nk
mark NOUN nk
> X nk
in ADP ROOT
der DET nk
< ADJ nk
mark NOUN nk
> X ROOT
Leopold PROPN pnc
  SPACE dep
Stadt NOUN ROOT
< ADJ nk
mark NOUN nk
> X punct
  SPACE dep
alt ADV mo
60 NUM nk
Jahr NOUN ROOT
Eva Reissenthallerin  ein Wittib in ihrem Hauß im <mark>Lerchenfeld<mark>  alt 95 Jahr 
Eva PROPN pnc
Reissenthallerin NOUN ROOT
  SPACE dep
ein DET nk
Wittib NOUN ROOT
in ADP mnr
ihrem DET nk
Hauß NOUN nk
im ADP mnr
< ADJ nk
mark NOUN nk
> X punct
Lerchenfeld PROPN pnc
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
alt ADV mo
95 NUM nk
Jahr NOUN ROOT
Georg Heidter  ein Holtz Aufleger im <mark>Copeinis  Hauß<mark> am <mark>Neustifft<mark>  alt 39 Jahr 
Georg PROPN pnc
Heidter PROPN ROOT
  SPACE dep
ein DET nk
Holtz NOUN ROOT
Aufleger ADJ nk
im ADP mnr
< ADJ nk
mark NOUN nk
> X punct
Copeinis PROPN ROOT
  SPACE dep
Hauß PROPN ROOT
< ADJ nk
mark NOUN mnr
> X cj
am ADP ROOT
< ADJ nk
mark NOUN nk
> X punct
Neustifft NOUN ROOT
< ADJ nk
mark NOUN nk
> X punct
  SPACE dep

Jungfrau Maria Theresia Heinrichin  in dem <mark>Gundl  Hof<mark>  alt 38 J 
Jungfrau NOUN ROOT
Maria PROPN pnc
Theresia PROPN pnc
Heinrichin PROPN pnc
  SPACE dep
in ADP mnr
dem DET nk
< ADJ nk
mark NOUN nk
> X punct
Gundl PROPN pnc
  SPACE dep
Hof PROPN ROOT
< ADJ nk
mark NOUN pnc
> X punct
  SPACE dep
alt ADV mo
38 NUM nk
J PROPN ROOT
Agnes Trefferin  Burgerl Wittwe  in dem <mark>Pfeifferischen Haus<mark>  am <mark>Graben<mark>  alt 46 J 
Agnes ADJ nk
Trefferin NOUN ROOT
  SPACE dep
Burgerl PROPN nk
Wittwe NOUN ROOT
  SPACE dep
in ADP mnr
dem DET nk
< ADJ nk
mark NOUN nk
> X nk
Pfeifferischen ADJ nk
Haus NOUN ROOT
< ADJ nk
mark NOUN nk
> X punct
  SPACE dep
am ADP mnr
< ADJ nk
mark NOUN nk
> X punct
Graben PROPN pnc
< ADJ nk
mark NOUN ROOT
> X punct
  SPACE dep
alt ADV mo
46 NUM nk
J PROPN ROOT
Johann Pittet  Burgerl Haar Puder Macher  in dem <mark>Nicoläer  Haus<mark>  im <mark>Fähndrich  Hof<mark>  alt 84 J 
Johann PROPN pnc
Pittet PROPN ROOT
  SPACE dep
Burgerl PROPN ROOT
Haar NO

Dem Herrn Johann Fridmann  Kaiserl , s K Johann  bey dem <mark>golden Adler<mark>  am <mark>Neuen Markt<mark>  alt 7 Viertel J 
Dem DET nk
Herrn NOUN da
Johann PROPN pnc
Fridmann PROPN nk
  SPACE dep
Kaiserl NOUN ROOT
, PUNCT punct
s PRON dep
K PROPN nk
Johann PROPN pnc
  SPACE dep
bey ADP mo
dem DET nk
< ADJ nk
mark NOUN nk
> X nk
golden ADJ nk
Adler NOUN ROOT
< ADJ nk
mark NOUN nk
> X punct
  SPACE dep
am ADP mnr
< ADJ nk
mark NOUN nk
> X punct
Neuen ADJ nk
Markt NOUN ROOT
< ADJ nk
mark NOUN nk
> X punct
  SPACE dep
alt ADV mo
7 NUM nmc
Viertel NOUN ROOT
J PROPN nk
Dem Martin Stark  Schreibern  s K Catharina  in dem <mark>Gariboldis   .<mark>  in der <mark>Weihburg<mark>  alt 3 J 
Dem DET nk
Martin PROPN pnc
Stark PROPN ROOT
  SPACE dep
Schreibern VERB ROOT
  SPACE dep
s X ep
K X uc
Catharina PROPN pnc
  SPACE dep
in ADP mnr
dem DET nk
< ADJ nk
mark NOUN nk
> X punct
Gariboldis PROPN ROOT
   SPACE dep
.<mark NOUN punct
> X ROOT
  SPACE dep
in ADP mnr
der DET nk
< ADJ nk
mark NOUN nk


KeyboardInterrupt: 

In [47]:
'''building Z Matrix, dimension: sentense x toponym or word x toponym?'''

df = pd.DataFrame(sterbeliste_words)

toponym = []
#create a counter = 0 ## just as controll
counter = 0

for row in df:
    if row in re.search(i):# Can i loop through my set of patterns?
            counter += 1
    for word in row:
        if re.search('mark'):
            toponym.append(1)
        elif word.isin(list(toponym_set)):
            if toponym[row] !=1:
                toponym +=1
    else:
        toponym = 0
        for word in row:
            word.strip('<mark>')

df['Toponym']= toponym



TypeError: search() missing 1 required positional argument: 'string'

In [24]:
'''Building T Matrix either with keyword or patterns'''

from scipy.sparse import coo_matrix

### with patterns 

T_train= coo_matrix((patterns), shape=(len(patterns),1))
 #not working yet



ValueError: inconsistent shapes: (10, 1) != (1, 1)

## 5. Building test data

In [26]:
'''Loading manually checked test data'''

with jsl.open('timemachine_evaluation_v1_edited_corrected.jsonl','r') as reader: #read in jsonl with jsonlines
    df = pd.json_normalize(reader) 
df
### I don't understand where the tagged toponym is or whether there is any

,text,_input_hash,_task_hash,tokens,_session_id,_view_id,spans,answer,meta.title,meta.div_id,meta.item_id
0,"Wenzel Milling., K. K. Mund=kuchel=thür=hüt., ...",1673446826,562968047,"[{'text': 'Wenzel', 'start': 0, 'end': 6, 'id'...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 49, 'end': 66, 'token_start': 16, '...",accept,Lista deren Verstorbenen zu Wien in = und vor ...,/db/apps/edoc/data/175x/1756/07/1756-07-07.xml,i29
1,Johann Carl / Strimpffstrickers Jung / im Thor...,-840701315,-552241706,"[{'text': 'Johann', 'start': 0, 'end': 6, 'id'...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 42, 'end': 62, 'token_start': 7, 't...",accept,Lista der Verstorbenen in der Stadt . | Den 1...,/db/apps/edoc/data/170x/1703/08/1703-08-17.xml,i15
2,"Dem Michael Brikner, Lakeyen, s. K. Regina, im...",-1342458309,115244989,"[{'text': 'Dem', 'start': 0, 'end': 3, 'id': 0...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 47, 'end': 65, 'token_start': 11, '...",accept,Lista deren Verstorbenen zu Wien in = und vor ...,/db/apps/edoc/data/174x/1746/02/1746-02-26.xml,i55
3,"Die Wol=edle Jungf. Eleon. Weichenbergerin, be...",-1344295137,-1995962992,"[{'text': 'Die', 'start': 0, 'end': 3, 'id': 0...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 54, 'end': 62, 'token_start': 12, '...",accept,Lista deren Verstorbenen zu Wien in = und vor ...,/db/apps/edoc/data/175x/1756/07/1756-07-07.xml,i11
4,"Barb Ofnerin, Wittwe, beym gold. Pelican am Sp...",-808830057,-1712038725,"[{'text': 'Barb', 'start': 0, 'end': 4, 'id': ...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 27, 'end': 40, 'token_start': 6, 't...",accept,Lista deren Verstorbenen zu Wien in = und vor ...,/db/apps/edoc/data/175x/1756/07/1756-07-07.xml,i32
...,...,...,...,...,...,...,...,...,...,...,...
156,Gottlieb Rabel / ein gewesterHaußmei=ster beyn...,257255689,-646239560,"[{'text': 'Gottlieb', 'start': 0, 'end': 8, 'i...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 47, 'end': 56, 'token_start': 8, 't...",accept,Lista aller Verstorbenen in und vor der Stadt ...,/db/apps/edoc/data/170x/1706/02/1706-02-03.xml,i54
157,Dem Math. Knib / Kaiserl. Reut=Knechten / s. K...,430814103,401245959,"[{'text': 'Dem', 'start': 0, 'end': 3, 'id': 0...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 65, 'end': 78, 'token_start': 17, '...",accept,Lista deren Verstorbenen zu Wien in und vor St...,/db/apps/edoc/data/172x/1729/04/1729-04-06.xml,i52
158,Dem Wohl=Edel=Gebornen Herrn Frantz Ant. v.Pis...,1546633707,-174228923,"[{'text': 'Dem', 'start': 0, 'end': 3, 'id': 0...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 120, 'end': 136, 'token_start': 35,...",accept,Lista deren Verstorbenen zu Wien in und vor de...,/db/apps/edoc/data/173x/1732/07/1732-07-09.xml,i33
159,"Jos. Dernberger, Burgl. Tischler, bey St. Anna...",-430482518,2067097700,"[{'text': 'Jos', 'start': 0, 'end': 3, 'id': 0...",timemachine_evaluation_v1-NRastinger,ner_manual,"[{'start': 38, 'end': 46, 'token_start': 9, 't...",accept,Lista deren Verstorbenen zu Wien in = und vor ...,/db/apps/edoc/data/176x/1766/04/1766-04-09.xml,i10
